In [1]:
!pip3 install pandas nba_api


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [51]:
from nba_api.stats.endpoints import teamdetails, commonplayerinfo


custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

# Only available after v1.1.0
# Proxy Support, Custom Headers Support, Timeout Support (in seconds)
player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)

In [4]:
from nba_api.stats.static import teams
# print(teams.get_teams())

team_list = []
i=0
for team in teams.get_teams():
    i+=1
    print(team['id'], team['full_name'])
    team_list.append([team['id'], team['full_name']])

print(i)


1610612737 Atlanta Hawks
1610612738 Boston Celtics
1610612739 Cleveland Cavaliers
1610612740 New Orleans Pelicans
1610612741 Chicago Bulls
1610612742 Dallas Mavericks
1610612743 Denver Nuggets
1610612744 Golden State Warriors
1610612745 Houston Rockets
1610612746 Los Angeles Clippers
1610612747 Los Angeles Lakers
1610612748 Miami Heat
1610612749 Milwaukee Bucks
1610612750 Minnesota Timberwolves
1610612751 Brooklyn Nets
1610612752 New York Knicks
1610612753 Orlando Magic
1610612754 Indiana Pacers
1610612755 Philadelphia 76ers
1610612756 Phoenix Suns
1610612757 Portland Trail Blazers
1610612758 Sacramento Kings
1610612759 San Antonio Spurs
1610612760 Oklahoma City Thunder
1610612761 Toronto Raptors
1610612762 Utah Jazz
1610612763 Memphis Grizzlies
1610612764 Washington Wizards
1610612765 Detroit Pistons
1610612766 Charlotte Hornets
30


In [3]:
def convert_json_string_to_json(json_string):
    try:
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"
    except Exception as e:
        return f"Unexpected error: {e}"

In [37]:
# get list of plays by gameid
from nba_api.stats.endpoints import playbyplayv3
import json


game_info_str = playbyplayv3.PlayByPlayV3(end_period=1, game_id="0022200605", start_period=1).get_json()
# game_json = convert_json_string_to_json(game_info_str)
# print(game_json["game"])
game


<class 'str'>


In [10]:
# get list of games by all teams in 2021-2022 season

from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playbyplayv3
import json
import pandas as pd

#one line python function to get the team boston celtics from game_json
# 1610612738
# celtics_id=0
# #one line python function to get the team boston celtics from the object game_json
# for team in team_list:
#     if team[1]=="Boston Celtics":
#         print(team[0])
#         celtics_id = team[0]


games = {}


for team in team_list:

    gameLog = teamgamelog.TeamGameLog(season="2021-22", season_type_all_star="Regular Season", team_id=team[0]).team_game_log.get_data_frame()

    print("Team: "+str(team[0]) + " " +str(team[1]))

    for i in range(gameLog.shape[0]):
        
        game = gameLog.iloc[i]
        if game.Team_ID not in games:
            games[game.Team_ID] = [game.Game_ID]
        else:
            games[game.Team_ID].append(game.Game_ID)
        # print(game)

# print(games)


Team: 1610612737 Atlanta Hawks
Team: 1610612738 Boston Celtics


KeyboardInterrupt: 

In [ ]:
gameLog = teamgamelog.TeamGameLog(season="2021-22", season_type_all_star="Regular Season", team_id=1610612738).team_game_log.get_data_frame()
print(convert_json_string_to_json( teamgamelog.TeamGameLog(season="2021-22", season_type_all_star="Regular Season", team_id=1610612738).team_game_log.get_json()))

In [ ]:
print(games)

In [95]:
from nba_api.stats.endpoints import videoevents
res = videoevents.VideoEvents(game_event_id = "67", game_id="0022200605").get_json()
print(res)

{"resource": "videoevents", "parameters": {"GameID": "0022200605", "GameEventID": 67}, "resultSets": {"Meta": {"videoUrls": [{"uuid": "4549dfbf-fde2-4dcc-8065-afade5ada267", "dur": null, "stt": null, "stp": null, "sth": null, "stw": null, "mtt": null, "mtp": null, "mth": null, "mtw": null, "ltt": null, "ltp": null, "lth": null, "ltw": null}]}, "playlist": [{"gi": "0022200605", "ei": 67, "y": 2023, "m": "01", "d": "09", "gc": "2023-01-09/NOPWAS", "p": 1, "dsc": "Gibson 1' Dunk (2 PTS) (Morris 4 AST)", "ha": "WAS", "va": "NOP", "hpb": 14, "hpa": 16, "vpb": 12, "vpa": 12, "pta": 0}]}}


In [1]:
# for every game in games.teamid, fetch playbyplayv3
# from nba_api.stats.endpoints import playbyplayv3
from playbyplayv3 import PlayByPlayV3
import ast
import json

i=0
playDict = {}
for team in team_list:

    game_list = games[team[0]]

    for gameID in game_list:
        # i+=1
        print("game#"+str(i)+": "+str(gameID))

        found = False
        i=0
        while((not found) or (i<5)):
            try:
                gamePBP = PlayByPlayV3(end_period=1, game_id=str(gameID), start_period=1).get_json() 
                found = True
            except Exception as e:
                print(e)
                i+=1


        # print(gamePBP)
        
        if gameID not in playDict:
            playDict[gameID] = gamePBP
        

    if i==0:
        print(i)
        break

# print(playDict)

    

NameError: name 'team_list' is not defined

In [30]:
playDictBackup = playDict

In [29]:
import ast
import json

playDict2 = eval(playDict)
print(playDict2)

TypeError: eval() arg 1 must be a string, bytes or code object